<a href="https://www.kaggle.com/code/yaaangzhou/icr-eda-modelling?scriptVersionId=138623811" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


# Imports

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from catboost import Pool, CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Load the data
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')


In [4]:
numerical_features = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ',
                      'BC', 'BD', 'BN', 'BP', 'BQ', 'BR', 'BZ',
                      'CB', 'CC', 'CD', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU', 'CW',
                      'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
                      'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU',
                      'FC', 'FD', 'FE', 'FI', 'FL', 'FR', 'FS',
                      'GB', 'GE', 'GF', 'GH', 'GI', 'GL']
categorical_features = ['EJ']
features = numerical_features + categorical_features

In [5]:
train['EJ'] = train['EJ'].replace({'A': 0, 'B': 1})
test['EJ'] = test['EJ'].replace({'A': 0, 'B': 1})

In [6]:
X=train.drop("Class",axis=1)
y=train["Class"]

X_test=test.copy()

In [7]:
X = X.drop('Id',axis = 1)

In [8]:
from IPython.display import display
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb

def obj_xgb(trial):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.001, 1.0),
        'objective': 'binary:logistic', 
        'eval_metric': 'logloss', 
    }

    # 构建XGBoost模型
    model = xgb.XGBClassifier(**params)
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 在验证集上进行预测
    y_pred = model.predict_proba(X_valid)[:, 1]  # 预测概率
    
    # 计算ROC AUC作为目标函数的评估指标
    roc_auc = roc_auc_score(y_valid, y_pred)
    
    return roc_auc

# 创建Study对象并进行优化
study = optuna.create_study(direction='maximize')
study.optimize(obj_xgb, n_trials=100)

# 显示Study结果
display(study)

print('Best trial:', study.best_trial.params)
print('Best ROC AUC:', study.best_trial.value)


[I 2023-08-01 22:02:01,774] A new study created in memory with name: no-name-4f0858e0-601c-45ec-9ab7-ddbdeed40c04
/tmp/ipykernel_20/2884987818.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
/tmp/ipykernel_20/2884987818.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
/tmp/ipykernel_20/2884987818.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` inst

Best trial: {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.0996726606561162, 'subsample': 0.5665283767660989, 'colsample_bytree': 0.8699114505283774, 'gamma': 0.004824624322246532}
Best ROC AUC: 0.9969866551872578


# Train best model

In [9]:
best_xgb = xgb.XGBClassifier(n_estimadors = 400,
                            max_depth = 5,
                            learning_rate = 0.0015383461504392903,
                            subsample = 0.8714696835501025,
                            colsample_bytree = 0.5506296722490095,
                            gamma = 0.7566166783103027)
best_xgb.fit(X,y)

[22:03:33] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimadors" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5506296722490095, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.7566166783103027, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.0015383461504392903, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimadors=400, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [10]:
pred = test.copy()
pred.drop('Id',axis = 1,inplace=True)
pred.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y_pred = best_xgb.predict_proba(pred)

In [12]:
submit = pd.DataFrame(y_pred,columns=['class_0','class_1'])
submit['Id'] = test["Id"]
submit.reindex(columns=['Id','class_0','class_1'])

,Id,class_0,class_1
0,00eed32682bb,0.514287,0.485713
1,010ebe33f668,0.514287,0.485713
2,02fa521e1838,0.514287,0.485713
3,040e15f562a2,0.514287,0.485713
4,046e85c7cc7f,0.514287,0.485713


In [13]:
'''from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
import optuna.integration.lightgbm as lgb
import optuna
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler




params = {
    'boosting_type': 'GBDT',
    'objective':"binary",
    'metric':'binary_logloss',
    'random_state': 42,
}



for i in range(20):
    
    positive_count_train = y.value_counts()[1]    
    sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 1: positive_count_train},random_state=i, replacement=True)
    X_re, y_re = sampler.fit_resample(X, y)

    (X_train, X_val , y_train , y_val) = train_test_split(X_re, y_re , test_size = 0.2,random_state=42)
    lgb_eval = lgb.Dataset(X_val.drop("Id",axis=1), y_val,free_raw_data=False) 
    lgb_train = lgb.Dataset(X_re.drop("Id",axis=1), y_re,free_raw_data=False)         
    model = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                      categorical_feature = categorical_features,
                      num_boost_round=1000,
                      early_stopping_rounds=20,
                      verbose_eval=10)
    
    pred = model.predict(X_test.drop("Id",axis=1),num_iteration=model.best_iteration)

    if i == 0:
        output = pd.DataFrame(pred, columns=['pred' + str(i + 1)])
        output2 = output
    else:
        output = pd.DataFrame(pred, columns=['pred' + str(i + 1)])
        output2 = pd.concat([output2, output], axis=1)

'''

'from imblearn.under_sampling import RandomUnderSampler\nimport numpy as np\nfrom sklearn.metrics import log_loss\nfrom sklearn.model_selection import KFold\nimport optuna.integration.lightgbm as lgb\nimport optuna\nimport pandas as pd\nimport numpy as np\nfrom sklearn.datasets import make_classification\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import roc_auc_score\nfrom xgboost import XGBClassifier\nfrom sklearn.ensemble import RandomForestClassifier\nfrom imblearn.under_sampling import RandomUnderSampler\n\n\n\n\nparams = {\n    \'boosting_type\': \'GBDT\',\n    \'objective\':"binary",\n    \'metric\':\'binary_logloss\',\n    \'random_state\': 42,\n}\n\n\n\nfor i in range(20):\n    \n    positive_count_train = y.value_counts()[1]    \n    sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 1: positive_count_train},random_state=i, replacement=True)\n    X_re, y_re = sampler.fit_resample(X, y)\n\n    (X_train, X_val , y_train , y_

In [14]:
'''pred = output2.mean(axis='columns')'''

"pred = output2.mean(axis='columns')"

In [15]:
'''submit=pd.DataFrame(test_df["Id"], columns=["Id"])'''

'submit=pd.DataFrame(test_df["Id"], columns=["Id"])'

In [16]:
'''submit["class_0"]=1-pred
submit["class_1"]=pred'''

'submit["class_0"]=1-pred\nsubmit["class_1"]=pred'

In [17]:
submit.to_csv('submission.csv',index=False)

In [18]:
submit

,class_0,class_1,Id
0,0.514287,0.485713,00eed32682bb
1,0.514287,0.485713,010ebe33f668
2,0.514287,0.485713,02fa521e1838
3,0.514287,0.485713,040e15f562a2
4,0.514287,0.485713,046e85c7cc7f
